## LangChain chains

In [1]:
!pip install -q langchain langchain-openai python-dotenv


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


### Simple chain: prompt → model → result

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a Python programming expert and an AI genius."),
    ("user", "Write a code related to {topic}")
])

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

chain = prompt | llm | StrOutputParser()
result = chain.invoke({'topic': 'AGI'})


In [3]:
print(result)

Creating a code snippet related to Artificial General Intelligence (AGI) is a complex task, as AGI refers to a type of AI that can understand, learn, and apply knowledge across a wide range of tasks, similar to human intelligence. However, I can provide a simplified example that demonstrates some foundational concepts that could be part of an AGI system, such as learning from experience and making decisions based on that learning.

Below is a Python code snippet that simulates a basic reinforcement learning agent. This agent learns to navigate a simple grid environment to reach a goal. While this is far from AGI, it illustrates some principles of learning and decision-making.

```python
import numpy as np
import random

class GridEnvironment:
    def __init__(self, size, goal):
        self.size = size
        self.goal = goal
        self.state = (0, 0)  # Start at the top-left corner

    def reset(self):
        self.state = (0, 0)
        return self.state

    def step(self, actio

### Sequential chain: 2 models in single sequence

In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 1) Chain: summary (input: {text} → output: str)
summary_chain = (
    ChatPromptTemplate.from_messages([
        ("system", "Summarize the text below in 1–2 sentences."),
        ("user", "{text}")
    ])
    | llm
    | StrOutputParser()
)

# 2) Adapter: replace str → {"text": str} and pass to next prompt
to_dict = RunnableLambda(lambda s: {"text": s})

# 3) Chain: translate summary to french (input: {text} → output: str)
translate_chain = (
    ChatPromptTemplate.from_messages([
        ("system", "Translate the text into French."),
        ("user", "{text}")
    ])
    | llm
    | StrOutputParser()
)

sequential_chain = summary_chain | to_dict | translate_chain
input_text = "LangChain enables the creation of AI applications by combining models, prompts, and tools into coherent pipelines."

final_translation = sequential_chain.invoke({"text": input_text})

print(final_translation)


LangChain facilite le développement d'applications d'IA en intégrant des modèles, des invites et des outils dans des flux de travail structurés.


### Branching chain: one response, two processing

In [5]:
from langchain_core.runnables import RunnableParallel

# summary prompt
prompt_summary = ChatPromptTemplate.from_template("Summarize: {text}")

# sentiment prompt
prompt_sentiment = ChatPromptTemplate.from_template("Classify sentiment: {text}")

branch_chain = RunnableParallel(
    summary=(prompt_summary | llm | StrOutputParser()),
    sentiment=(prompt_sentiment | llm | StrOutputParser())
)

text = "I am very happy with this course, I learned a lot about LangChain! Now I know that LangChain enables the creation of AI applications by combining models, prompts, and tools into coherent pipelines."
result = branch_chain.invoke({"text": text})

print(result)


{'summary': 'The course was highly beneficial, providing valuable insights into LangChain, which facilitates the development of AI applications by integrating models, prompts, and tools into cohesive pipelines.', 'sentiment': 'The sentiment of the statement is positive. The speaker expresses happiness and satisfaction with the course and indicates that they have gained valuable knowledge.'}
